In [0]:
# If you wish to see what's inside the spark object and the sparkContext run these commands
# but your code will work regardless since they have already been set up.
spark
sc = spark.sparkContext

In [0]:
from pyspark.sql import functions as F # This will load the class where spark sql functions are contained
from pyspark.sql import Row # this will let us manipulate rows with spark sql
from pyspark.sql.types import * # Import types to convert columns using spark sql
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import explode
from pyspark.sql.functions import col, sum, desc, row_number
from pyspark.sql.functions import count
from pyspark.sql.functions import to_date, to_timestamp
from pyspark.sql.window import Window
from pyspark.sql.functions import when
from pyspark.sql.functions import expr

## <div style="text-align: center;text-decoration: underline;">EDA<br><br> ![image eda](https://ppcexpo.com/blog/wp-content/uploads/2022/01/exploratory-data-analysis.jpg) </div>

In [0]:
# Créez un DataFrame à partir des données
steam = spark.read.option("multiline", "true").json("s3://full-stack-bigdata-datasets/Big_Data/Project_Steam/steam_game_output.json")

In [0]:
steam.show()
steam.printSchema()

+--------------------+-------+
|                data|     id|
+--------------------+-------+
|{10, [Multi-playe...|     10|
|{1000000, [Single...|1000000|
|{1000010, [Single...|1000010|
|{1000030, [Multi-...|1000030|
|{1000040, [Single...|1000040|
|{1000080, [Multi-...|1000080|
|{1000100, [Single...|1000100|
|{1000110, [Multi-...|1000110|
|{1000130, [Single...|1000130|
|{1000280, [Single...|1000280|
|{1000310, [Multi-...|1000310|
|{1000360, [Multi-...|1000360|
|{1000370, [Single...|1000370|
|{1000380, [Single...|1000380|
|{1000410, [Single...|1000410|
|{1000470, [Single...|1000470|
|{1000480, [Single...|1000480|
|{1000500, [Multi-...|1000500|
|{1000510, [], 0, ...|1000510|
|{1000540, [Multi-...|1000540|
+--------------------+-------+
only showing top 20 rows

root
 |-- data: struct (nullable = true)
 |    |-- appid: long (nullable = true)
 |    |-- categories: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- ccu: long (nullable = true)
 |    |-- de

In [0]:

# Liste des colonnes à extraire
columns_to_extract = ['genre', 'publisher', 'platforms','positive','negative','categories','name','developer','discount','initialprice','ccu','release_date','languages','required_age']

# Boucle à travers chaque colonne et appliquez getField() pour extraire la valeur
for column in columns_to_extract:
    steam = steam.withColumn(column, col("data").getField(column))

In [0]:
# Créer une vue temporaire pour pouvoir utiliser SQL
steam.createOrReplaceTempView("table_temporaire")

# Exécuter une requête SQL pour compter le nombre de valeurs vides dans une colonne spécifique
resultatc = spark.sql("SELECT COUNT(CASE WHEN release_date IS NULL OR release_date = '' THEN 1 END) AS nombre_valeurs_vides FROM table_temporaire")

# Afficher le nombre de valeurs vides dans la colonne spécifique
resultatc.show()



+--------------------+
|nombre_valeurs_vides|
+--------------------+
|                  99|
+--------------------+



In [0]:
# Utilisez la fonction cast() pour convertir la colonne string en INTEGER
dfint = steam.withColumn("price", col("initialprice").cast("INTEGER"))

# Afficher le DataFrame résultant
dfint.show()

dfint.printSchema()

+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+
|                data|     id|               genre|                   publisher|           platforms|positive|negative|          categories|                                name|                   developer|discount|initialprice|  ccu|release_date|           languages|required_age|price|
+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+
|{10, [Multi-playe...|     10|              Action|                       Valve|  {true, true, true}|  201215|    5199|[Multi-player, Va

In [0]:
# Utilisez la fonction cast() pour convertir la colonne string en INTEGER
dfint1 = dfint.withColumn("promo", col("discount").cast("INTEGER"))

# Afficher le DataFrame résultant
dfint1.show()

dfint1.printSchema()

+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+-----+
|                data|     id|               genre|                   publisher|           platforms|positive|negative|          categories|                                name|                   developer|discount|initialprice|  ccu|release_date|           languages|required_age|price|promo|
+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+-----+
|{10, [Multi-playe...|     10|              Action|                       Valve|  {true, true, true}|  201215|    5199

In [0]:
# Utilisez la fonction cast() pour convertir la colonne string en INTEGER
dfint2 = dfint1.withColumn("pegi", col("required_age").cast("INTEGER"))

# Afficher le DataFrame résultant
dfint2.show()

dfint2.printSchema()

+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+-----+----+
|                data|     id|               genre|                   publisher|           platforms|positive|negative|          categories|                                name|                   developer|discount|initialprice|  ccu|release_date|           languages|required_age|price|promo|pegi|
+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+-----+----+
|{10, [Multi-playe...|     10|              Action|                       Valve|  {true, true, true}|  

In [0]:
# Utilisez la fonction cast() pour convertir la colonne string en INTEGER
dfint3 = dfint2.withColumn("reviewsgood", col("positive").cast("INTEGER"))

# Afficher le DataFrame résultant
dfint3.show()

dfint3.printSchema()

+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+-----+----+-----------+
|                data|     id|               genre|                   publisher|           platforms|positive|negative|          categories|                                name|                   developer|discount|initialprice|  ccu|release_date|           languages|required_age|price|promo|pegi|reviewsgood|
+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+-----+----+-----------+
|{10, [Multi-playe...|     10|              Action|                

In [0]:
# Utilisez la fonction cast() pour convertir la colonne string en INTEGER
dfint4 = dfint3.withColumn("reviewsneg", col("negative").cast("INTEGER"))

# Afficher le DataFrame résultant
dfint4.show()

dfint4.printSchema()

+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+-----+----+-----------+----------+
|                data|     id|               genre|                   publisher|           platforms|positive|negative|          categories|                                name|                   developer|discount|initialprice|  ccu|release_date|           languages|required_age|price|promo|pegi|reviewsgood|reviewsneg|
+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+-----+----+-----------+----------+
|{10, [Multi-playe...|     10|    

In [0]:
from pyspark.sql.functions import to_date

# Suppose df est votre DataFrame contenant les données
# Utilisez la fonction to_date() pour convertir la chaîne en date
dftets = dfint4.withColumn("date_column", to_date(steam["release_date"], "yyyy/MM/d"))

In [0]:
from pyspark.sql.functions import year, col

# Suppose df est votre DataFrame contenant les données
# Utilisez la fonction year() pour extraire l'année à partir de la colonne release_date
dftt = dftets.withColumn("year", year(col("date_column")))

In [0]:
dftt.show()

+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+-----+----+-----------+----------+-----------+----+
|                data|     id|               genre|                   publisher|           platforms|positive|negative|          categories|                                name|                   developer|discount|initialprice|  ccu|release_date|           languages|required_age|price|promo|pegi|reviewsgood|reviewsneg|date_column|year|
+--------------------+-------+--------------------+----------------------------+--------------------+--------+--------+--------------------+------------------------------------+----------------------------+--------+------------+-----+------------+--------------------+------------+-----+-----+----+-----------+----------+-

In [0]:
# Supprimez les colonnes non nécessaires
columns_to_drop = ['data', 'id','release_date','initialprice','discount','required_age','positive','negative']  # Ajoutez les colonnes à supprimer ici
steam = dftt.drop(*columns_to_drop)

# Affichez le DataFrame résultant
steam.show()

+--------------------+----------------------------+--------------------+--------------------+------------------------------------+----------------------------+-----+--------------------+-----+-----+----+-----------+----------+-----------+----+
|               genre|                   publisher|           platforms|          categories|                                name|                   developer|  ccu|           languages|price|promo|pegi|reviewsgood|reviewsneg|date_column|year|
+--------------------+----------------------------+--------------------+--------------------+------------------------------------+----------------------------+-----+--------------------+-----+-----+----+-----------+----------+-----------+----+
|              Action|                       Valve|  {true, true, true}|[Multi-player, Va...|                      Counter-Strike|                       Valve|13990|English, French, ...|  999|    0|   0|     201215|      5199| 2000-11-01|2000|
|Action, Adventure...|  

## <div style="text-align: center;text-decoration: underline;">Analyse au niveau « macro »<br><br> ![Analyse au niveau « macro »](https://eminence.ch/wp-content/uploads/2023/09/Data_Analytics_2-3.png) </div> 

In [0]:
#####  Y a-t-il des années avec plus de sorties ? Y a-t-il eu plus ou moins de sorties de jeux pendant le Covid par exemple ?

# Utilisez groupBy() pour grouper par une colonne et appliquer une opération d'agrégation
resultannee = steam.groupBy("year").agg(count("*").alias("count_year"))

# Triez les résultats par ordre décroissant
resultannee = resultannee.orderBy("count_year", ascending=False)

# Affichez le résultat
resultannee.show()


+----+----------+
|year|count_year|
+----+----------+
|2021|      8805|
|2020|      8287|
|2018|      7663|
|2022|      7451|
|2019|      6949|
|2017|      6006|
|2016|      4176|
|2015|      2566|
|2014|      1550|
|2013|       469|
|2012|       344|
|2009|       309|
|2010|       281|
|2011|       267|
|null|       222|
|2008|       159|
|2007|        98|
|2006|        61|
|2004|         6|
|2005|         6|
+----+----------+
only showing top 20 rows



In [0]:
display(resultannee)

year,count_year
2021,8805
2020,8287
2018,7663
2022,7451
2019,6949
2017,6006
2016,4176
2015,2566
2014,1550
2013,469


Databricks visualization. Run in Databricks to view.

In [0]:
#####  Quel éditeur a sorti le plus de jeux sur Steam ?

# Utilisez groupBy() pour grouper par une colonne et appliquer une opération d'agrégation
result = steam.groupBy("publisher").agg(count("*").alias("count"))

# Triez les résultats par ordre décroissant
result = result.orderBy("count", ascending=False)

# Affichez le résultat
result.show()


+--------------------+-----+
|           publisher|count|
+--------------------+-----+
|      Big Fish Games|  422|
|              8floor|  202|
|                SEGA|  165|
|      Strategy First|  151|
|         Square Enix|  141|
|     Choice of Games|  140|
|       Sekai Project|  132|
|            HH-Games|  132|
|                    |  132|
|             Ubisoft|  127|
|        Laush Studio|  126|
|          THQ Nordic|  125|
|Alawar Entertainment|  107|
|  Fulqrum Publishing|  104|
|     Plug In Digital|  101|
|            Ziggurat|  100|
|     Slitherine Ltd.|   99|
|   Sokpop Collective|   99|
|    Devolver Digital|   98|
|KOEI TECMO GAMES ...|   90|
+--------------------+-----+
only showing top 20 rows



In [0]:
#####  Quels sont les jeux les mieux notés ?

# Utilisez groupBy() pour grouper par une colonne et appliquer une opération d'agrégation
result1 = steam.groupBy("name", "reviewsgood").agg(count("*"))

# Triez les résultats par ordre décroissant
result1 = result1.orderBy("reviewsgood", ascending=False)

# Affichez le résultat
result1.show()

+--------------------+-----------+--------+
|                name|reviewsgood|count(1)|
+--------------------+-----------+--------+
|Counter-Strike: G...|    5943345|       1|
|              Dota 2|    1534895|       1|
|  Grand Theft Auto V|    1229265|       1|
| PUBG: BATTLEGROUNDS|    1185361|       1|
|            Terraria|    1014711|       1|
|Tom Clancy's Rain...|     942910|       1|
|         Garry's Mod|     861240|       1|
|     Team Fortress 2|     846407|       1|
|                Rust|     732513|       1|
|       Left 4 Dead 2|     643836|       1|
|The Witcher 3: Wi...|     632627|       1|
|            Among Us|     586302|       1|
|Euro Truck Simula...|     572368|       1|
|    Wallpaper Engine|     561096|       1|
|            PAYDAY 2|     532013|       1|
|    Dead by Daylight|     509637|       1|
|      Stardew Valley|     497558|       1|
|       Rocket League|     496499|       1|
|          ELDEN RING|     490203|       1|
|ARK: Survival Evo...|     48131

In [0]:
#####  Comment sont distribués les prix ?

# Calculer la distribution des prix en termes de fréquence
result2 = steam.groupBy("price").agg(count("*").alias("count"))
                                                              
# Triez
distribution_prix = result2.orderBy("count", ascending=False)                 

# Afficher le résultat
distribution_prix.show()

+-----+-----+
|price|count|
+-----+-----+
|    0| 7780|
|  499| 6518|
|  999| 6336|
|   99| 5441|
|  199| 4285|
|  299| 3738|
| 1499| 3027|
| 1999| 2992|
|  399| 2591|
|  699| 1964|
|  599| 1926|
|  799| 1631|
| 2499|  936|
| 2999|  906|
| 1299|  851|
| 1199|  850|
|  899|  756|
| 1099|  460|
| 3999|  447|
| 1799|  297|
+-----+-----+
only showing top 20 rows



In [0]:
display(distribution_prix)

price,count
0,7780
499,6518
999,6336
99,5441
199,4285
299,3738
1499,3027
1999,2992
399,2591
699,1964


In [0]:
#####  Y a-t-il beaucoup de jeux avec une réduction ?  "discount"

# Calculer la distribution des prix en termes de fréquence
result3 = steam.groupBy("promo").agg(count("*").alias("count"))
                                                              
# Triez
distribution_discount = result3.orderBy("count", ascending=False)                 

# Afficher le résultat
distribution_discount.show()

# Nom de la colonne pour laquelle vous voulez compter les éléments
column_name = "promo" # Remplacer "votre_colonne" par le nom de votre colonne

# Filtrer les éléments différents de zéro dans la colonne spécifiée
filtered = steam.filter(steam[column_name] != 0)

# Compter le nombre d'éléments différents de zéro dans la colonne spécifiée
count_elements = filtered.select(column_name).count()

# Afficher le nombre d'éléments différents de zéro dans la colonne spécifiée
print(f"Nombre de jeux en promotion '{column_name}': {count_elements}")

+-----+-----+
|promo|count|
+-----+-----+
|    0|53173|
|   50|  350|
|   90|  239|
|   80|  228|
|   75|  223|
|   40|  164|
|   51|  146|
|   70|  137|
|   60|  137|
|   30|  131|
|   20|  112|
|   25|   85|
|   10|   53|
|   15|   45|
|   35|   39|
|   65|   37|
|   87|   36|
|   85|   34|
|   83|   26|
|   48|   25|
+-----+-----+
only showing top 20 rows

Nombre de jeux en promotion 'promo': 2518


In [0]:
#####  Quelles sont les langues les plus représentées ?

# Diviser la colonne languages en une liste de langues, puis utiliser explode pour obtenir une ligne par langue
df_lang = steam.withColumn("language", explode(split(steam["languages"], ",\s*")))

# Compter le nombre d'occurrences de chaque langue et trier les résultats par ordre décroissant
lang_counts = df_lang.groupBy("language").count().orderBy(desc("count"))

# Afficher les langues les plus représentées
print("Langues les plus représentées :")
lang_counts.show()

Langues les plus représentées :
+--------------------+-----+
|            language|count|
+--------------------+-----+
|             English|55116|
|              German|14019|
|              French|13426|
|             Russian|12922|
|  Simplified Chinese|12782|
|     Spanish - Spain|12233|
|            Japanese|10368|
|             Italian| 9304|
| Portuguese - Brazil| 6750|
|              Korean| 6600|
| Traditional Chinese| 6307|
|              Polish| 5376|
|Portuguese - Port...| 4011|
|             Turkish| 3602|
|               Dutch| 3080|
|Spanish - Latin A...| 2732|
|               Czech| 2352|
|             Swedish| 2047|
|           Ukrainian| 1928|
|           Hungarian| 1923|
+--------------------+-----+
only showing top 20 rows



In [0]:
#####  Existe-t-il de nombreux jeux interdits aux moins de 16/18 ans ?

# Calculer la distribution des prix en termes de fréquence
result4 = steam.groupBy("pegi").agg(count("*").alias("count"))
                                                              
# Triez
distribution_pegi = result4.orderBy("count", ascending=False)                 

# Afficher le résultat
distribution_pegi.show()

# Filtrer les lignes où la valeur de la colonne "pegi" est entre 16 et 18 inclusivement
jeux_interdits = distribution_pegi.filter((distribution_pegi["pegi"] >= 16) & (distribution_pegi["pegi"] <= 18)).agg({"count": "sum"}).collect()[0][0]

# Afficher le nombre d'éléments entre 16 et 18 inclusivement
print(f"Nombre d'éléments entre 16 et 18 inclusivement : {jeux_interdits}")

+----+-----+
|pegi|count|
+----+-----+
|   0|55030|
|  15|  264|
|  18|  223|
|  16|   38|
|  17|   38|
|  12|   32|
|  13|   26|
|  14|   10|
|  10|    7|
|   6|    4|
| 180|    4|
|null|    3|
|   3|    3|
|   8|    3|
|   7|    2|
|  20|    1|
|   5|    1|
|   9|    1|
|  35|    1|
+----+-----+

Nombre d'éléments entre 16 et 18 inclusivement : 299


<div style="text-align: center;text-decoration: underline;font-weight: 700;font-size: 1.75em;line-height: 1.225;">Analyse des genres<br><br>
    <img src="https://www.games-geeks.fr/wp-content/uploads/2023/12/sorties-jeux-video-decembre-2023.jpg" style="width: 50%;" alt="Analyse des genres">
</div>


In [0]:
#####  Quels sont les genres les plus représentés ?

# Diviser la colonne languages en une liste de langues, puis utiliser explode pour obtenir une ligne par langue
df_genres = steam.withColumn("genre_games", explode(split(steam["genre"], ",\s*")))

# Compter le nombre d'occurrences de chaque langue et trier les résultats par ordre décroissant
genres_counts = df_genres.groupBy("genre_games").count().orderBy(desc("count"))

# Afficher les langues les plus représentées
print("Les genres de jeux les plus représentées :")
genres_counts.show()

Les genres de jeux les plus représentées :
+--------------------+-----+
|         genre_games|count|
+--------------------+-----+
|               Indie|39681|
|              Action|23759|
|              Casual|22086|
|           Adventure|21431|
|            Strategy|10895|
|          Simulation|10836|
|                 RPG| 9534|
|        Early Access| 6145|
|        Free to Play| 3393|
|              Sports| 2666|
|              Racing| 2155|
|Massively Multipl...| 1460|
|           Utilities|  682|
|Design & Illustra...|  406|
|Animation & Modeling|  322|
|           Education|  317|
|    Video Production|  247|
|    Audio Production|  195|
|             Violent|  168|
|   Software Training|  164|
+--------------------+-----+
only showing top 20 rows



In [0]:
#####  Existe-t-il des genres qui ont un meilleur ratio d'avis positifs/négatifs ?

# Calculer le ratio des genres avec les reviews positis et négatifs
ratio_positifs_negatifs = df_genres.select(
    col("genre_games"),
    (col("reviewsgood") / col("reviewsneg")).alias("ratio_positifs_negatifs")
)

# Trier le résultat par ordre décroissant du ratio
ratio_positifs_negatifs = ratio_positifs_negatifs.orderBy(col("ratio_positifs_negatifs").desc())

# Afficher le résultat
ratio_positifs_negatifs.show()

# Calculer la distribution des ratio par genres
positifs_negatifs = ratio_positifs_negatifs.groupBy("genre_games").agg(count("*").alias("count"))

# Trier le résultat par ordre décroissant
positifs_negatifs = positifs_negatifs.orderBy(col("count").desc())

print("Voici les genres avec les meilleurs ration :")
positifs_negatifs.show()

+------------+-----------------------+
| genre_games|ratio_positifs_negatifs|
+------------+-----------------------+
|      Casual|                  937.0|
|      Action|                  264.0|
|       Indie|                  264.0|
|      Casual|                  252.0|
|Early Access|                  252.0|
|      Casual|                  247.0|
|      Action|                  227.0|
|   Adventure|                  227.0|
|       Indie|                  227.0|
|       Indie|                  226.0|
|   Adventure|                  226.0|
|         RPG|                  219.0|
|   Adventure|                  203.5|
|       Indie|                  203.5|
|       Indie|                  196.0|
|   Adventure|                  196.0|
|      Action|                  196.0|
|   Adventure|                  196.0|
|       Indie|                  196.0|
|      Casual|                  192.0|
+------------+-----------------------+
only showing top 20 rows

Voici les genres avec les meilleurs ra

In [0]:
#####  Certains éditeurs ont-ils des genres favoris ?

# Définir la fenêtre de partition par éditeur et trier par le nombre d'occurrences de chaque genre
windowSpec = Window.partitionBy("publisher").orderBy(desc("count"))

# Ajouter une colonne de rang pour chaque genre par éditeur
df_with_rank = df_genres.groupBy("publisher", "genre_games").count().withColumn("rank", row_number().over(windowSpec))

# Filtrer pour ne conserver que le genre avec le rang 1 pour chaque éditeur (le genre le plus représenté)
genre_plus_represente_par_editeur = df_with_rank.filter(col("rank") == 1).drop("count", "rank")

# Afficher le résultat
genre_plus_represente_par_editeur.show()

+--------------------+------------+
|           publisher| genre_games|
+--------------------+------------+
|           AK Studio|       Indie|
|         Alon Zubina|       Indie|
|        Appnori Inc.|Early Access|
| Art Games Studio...|      Casual|
|      Asterion Games|       Indie|
|           BBB Games|       Indie|
|   Beach Day Studios|       Indie|
|               CGevo|   Adventure|
| Dipl. Ing. Rolf-...|  Simulation|
|        FurGoldGames|      Action|
|     Game for people|      Sports|
|  Genuine Studio Ltd|   Adventure|
| H2 Interactive C...|   Adventure|
| Hangover Cat Pur...|      Casual|
|             Iful GS|       Indie|
| Indevelopment Pr...|Early Access|
|            MV Games|   Adventure|
|           Metal Fox|       Indie|
| Paper Bunker s.r.o.|      Racing|
|      PeriScope Game|      Casual|
+--------------------+------------+
only showing top 20 rows



In [0]:
#####  Quels sont les genres les plus lucratifs ?

# Calculer le revenu total généré par chaque genre
revenu_par_genre = df_genres.groupBy("genre_games").agg(sum("price").alias("revenu_total"))

# Trier le résultat par ordre décroissant du revenu total
genres_lucratifs = revenu_par_genre.orderBy(col("revenu_total").desc())

# Afficher le résultat
genres_lucratifs.show()

+--------------------+------------+
|         genre_games|revenu_total|
+--------------------+------------+
|               Indie|    26979710|
|              Action|    18954145|
|           Adventure|    17802060|
|              Casual|    12816307|
|          Simulation|    10164599|
|            Strategy|     9440421|
|                 RPG|     8928394|
|        Early Access|     5452918|
|              Sports|     2468279|
|              Racing|     1826089|
|           Utilities|      798741|
|Design & Illustra...|      793390|
|Massively Multipl...|      759845|
|Animation & Modeling|      615387|
|    Video Production|      489352|
|           Education|      462432|
|    Audio Production|      405198|
|    Game Development|      339612|
|   Software Training|      323893|
|       Photo Editing|      214405|
+--------------------+------------+
only showing top 20 rows



<div style="text-align: center;text-decoration: underline;font-weight: 700;font-size: 1.75em;line-height: 1.225;">Analyse de la plateforme<br><br>
    <img src="https://cdn.mos.cms.futurecdn.net/T3ZiBaJpMbQ64uodndXQ8Q-1200-80.jpg" style="width:40%;" alt="Analyse de la plateforme">
</div>

In [0]:
#####  La plupart des jeux sont-ils plutôt disponibles sur Windows/Mac/Linux ?

# Séparation de la colonne platforms
df = df_genres.withColumn("linux", col("platforms.linux"))
df = df.withColumn("mac", col("platforms.mac"))
df = df.withColumn("windows", col("platforms.windows"))

# Suppression de la colonne platforms d'origine si nécessaire
df = df.drop("platforms","genre")

# Afficher le résultat
df.show()

# Remplacer les valeurs True/False par Oui/Non dans les colonnes linux, mac et windows
df = df.withColumn("linux", when(col("linux"), "oui").otherwise("non"))
df = df.withColumn("mac", when(col("mac"), "oui").otherwise("non"))
df = df.withColumn("windows", when(col("windows"), "oui").otherwise("non"))

# Afficher le résultat
df.show()

# Calculer le nombre de "oui" dans les colonnes linux, mac et windows
count_linux = df.filter(col("linux") == "oui").count()
count_mac = df.filter(col("mac") == "oui").count()
count_windows = df.filter(col("windows") == "oui").count()


print("Voici le nombres de jeux disponible par plateforme :")

print("Linux:", count_linux)
print("Mac:", count_mac)
print("Windows:", count_windows)

+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----+-----+----+-----------+----------+-----------+----+-----------+-----+-----+-------+
|           publisher|          categories|                name|           developer|  ccu|           languages|price|promo|pegi|reviewsgood|reviewsneg|date_column|year|genre_games|linux|  mac|windows|
+--------------------+--------------------+--------------------+--------------------+-----+--------------------+-----+-----+----+-----------+----------+-----------+----+-----------+-----+-----+-------+
|               Valve|[Multi-player, Va...|      Counter-Strike|               Valve|13990|English, French, ...|  999|    0|   0|     201215|      5199| 2000-11-01|2000|     Action| true| true|   true|
|PsychoFlux Entert...|[Single-player, P...|           ASCENXION|IndigoBlue Game S...|    0|English, Korean, ...|  999|    0|   0|         27|         5| 2021-05-14|2021|     Action|false|false